In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
from pyproj import Proj, transform
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State



In [ ]:
file_path = 'H:/My Drive/DAE/CVA/Project 3/1.08_Crash_Data_Report_(detail).csv'
crash_data = pd.read_csv(file_path)
crash_data

In [ ]:
crash_data.info()

# Preprocessing

In [ ]:
crash_data['Age_Drv1'].fillna(crash_data['Age_Drv1'].mean(), inplace=True)

In [ ]:
crash_data.drop(['X', 'Y'], axis=1, inplace=True)

In [ ]:
crash_data_sorted = crash_data.sort_values(by='DateTime')
crash_data['DateTime'] = pd.to_datetime(crash_data['DateTime'])
crash_data['Date'] = crash_data['DateTime'].dt.date
crash_data['Month'] = crash_data['DateTime'].dt.month
crash_data['Hour'] = crash_data['DateTime'].dt.hour
crash_data['Lightcondition'] = crash_data['Lightcondition'].replace(['51', 'Unknown 51'], 'Unknown')  

In [ ]:
crash_data['Totalfatalities'].fillna(0, inplace=True)
crash_data['Totalinjuries'].fillna(0, inplace=True)

In [ ]:

crash_data['NormalizedInjuries'] = crash_data['Totalinjuries'] / crash_data['Totalinjuries'].max() * 20

# Visualizations

In [ ]:
incidents_by_year = crash_data.groupby('Year').size()
crash_data_sorted = crash_data.sort_values(by='DateTime')

In [ ]:
incidents_by_year.plot(kind='bar')
plt.title('Number of Incidents by Year')
plt.xlabel('Year')
plt.ylabel('Number of Incidents')
plt.show()

In [ ]:
date_counts = crash_data.groupby('Date').size().reset_index(name='Counts')

fig = px.line(date_counts,
              x='Date',
              y='Counts',
              title='Trend of Crashes Over Time')

fig.update_layout(xaxis_title='Date',
                  yaxis_title='Number of Crashes')

fig.show()


In [ ]:
crash_data.plot(kind='scatter', x='Longitude', y='Latitude', alpha=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geospatial Distribution of Crashes')
plt.show()

In [ ]:
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

In [ ]:
fig = px.scatter_mapbox(crash_data,
                        lat='Latitude',
                        lon='Longitude',
                        color='Injuryseverity', 
                        hover_name='StreetName',  
                        hover_data=['DateTime', 'Totalinjuries', 'Totalfatalities'], 
                        center={"lat": crash_data['Latitude'].mean(), "lon": crash_data['Longitude'].mean()},  
                        zoom=10,  
                        mapbox_style="open-street-map",  
                        title="Geospatial Distribution of Crashes by Injury Severity")  
fig.update_layout(width=1000, height=1100)

fig.show()


In [ ]:
fig = px.scatter_mapbox(crash_data,
                        lat='Latitude',
                        lon='Longitude',
                        color='Injuryseverity',
                        size='Totalinjuries',
                        size_max=15,
                        opacity=0.8,
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        hover_name='StreetName',
                        hover_data=['DateTime', 'Totalinjuries', 'Totalfatalities'],
                        center={"lat": crash_data['Latitude'].mean(), "lon": crash_data['Longitude'].mean()},
                        zoom=11,
                        mapbox_style="open-street-map",
                        animation_frame='Year',  
                        title="Geospatial Distribution of Crashes by Injury Severity")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(width=1000, height=1100)

fig.show()


In [ ]:
heatmap = go.Densitymapbox(lat=crash_data['Latitude'], lon=crash_data['Longitude'],
                           z=crash_data['Totalinjuries'], radius=10)

fatal_crashes = crash_data[crash_data['Injuryseverity'] == 'Fatal']
fatal_trace = go.Scattermapbox(lat=fatal_crashes['Latitude'], lon=fatal_crashes['Longitude'],
                               mode='markers', marker=go.scattermapbox.Marker(size=12, color='red', symbol='cross'))

fig = go.Figure([heatmap, fatal_trace])

fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=dict(lat=crash_data['Latitude'].mean(),
                                          lon=crash_data['Longitude'].mean()), zoom=10))

fig.update_layout(mapbox_layers=[
    {
        "below": 'traces',
        "sourcetype": "raster",
        "sourceattribution": "United States Geological Survey",
        "source": [
            "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
        ]
    }
])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(width=1000, height=1100)
fig.show()


In [ ]:
crash_trace = go.Scattermapbox(
    lat=crash_data['Latitude'],
    lon=crash_data['Longitude'],
    mode='markers',
    marker=dict(
        size=crash_data['NormalizedInjuries'],
        color=crash_data['Totalinjuries'],
        colorscale='Viridis',
        showscale=True,
        opacity=0.7
    ),
    text=crash_data['StreetName'],
    hoverinfo='text'
)

layout = go.Layout(
    mapbox=dict(
        center=dict(lat=crash_data['Latitude'].mean(), lon=crash_data['Longitude'].mean()),
        zoom=10,
        style='open-street-map'
    ),
    width= 900,
    height=1200,
    margin=dict(t=0, b=0, l=0, r=0)
)

fig = go.Figure(data=[crash_trace], layout=layout)

fig.show()


In [ ]:
app = dash.Dash(__name__)

fig = px.scatter_mapbox(crash_data,
                        lat='Latitude',
                        lon='Longitude',
                        hover_name='StreetName',
                        hover_data=['DateTime', 'Totalinjuries', 'Totalfatalities', 'JunctionRelation'],
                        mapbox_style="open-street-map",
                        width=1000,
                        height=1200)

app.layout = html.Div([
    dcc.Graph(
        id='crash-map',
        figure=fig
    ),
    html.Pre(id='click-data', style={'paddingTop': 35})
])

@app.callback(
    Output('click-data', 'children'),
    [Input('crash-map', 'clickData')])
def display_click_data(clickData):
    if clickData is not None:
        index = clickData['points'][0]['pointIndex']
        return generate_report_card(index)

def generate_report_card(index):
    data = crash_data.iloc[index]  
    return f"Date/Time: {data['DateTime']}\n"\
           f"Location: {data['StreetName']} at {data['CrossStreet']}\n"\
           f"Total Injuries: {data['Totalinjuries']}\n"\
           f"Total Fatalities: {data['Totalfatalities']}"

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  



In [ ]:
severity_counts = crash_data['Injuryseverity'].value_counts().reset_index()
severity_counts.columns = ['Injury Severity', 'Count']

fig = px.bar(severity_counts,
             x='Injury Severity',
             y='Count',
             title='Total Injuries by Severity')

fig.update_layout(xaxis_title='Injury Severity',
                  yaxis_title='Number of Injuries',
                  xaxis={'categoryorder':'total descending'})

fig.show()

In [ ]:
fig = px.scatter_mapbox(crash_data, lat='Latitude', lon='Longitude',
                        size='Totalfatalities', color='Totalfatalities',
                        color_continuous_scale=px.colors.sequential.Viridis,
                        size_max=15, zoom=10, mapbox_style="open-street-map")

fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=dict(lat=crash_data['Latitude'].mean(),
                                          lon=crash_data['Longitude'].mean()), zoom=10),
                  margin={"r":0,"t":0,"l":0,"b":0},
                 width = 1000, 
                 height = 1200)

fig.show()

In [ ]:
fig = px.scatter_mapbox(crash_data,
                        lat='Latitude',
                        lon='Longitude',
                        color='Lightcondition',
                        hover_name='StreetName',
                        hover_data=['DateTime', 'Totalinjuries', 'Totalfatalities'],
                        mapbox_style="open-street-map",
                        zoom=10,
                        title="Crashes by Light Condition")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(width=1000, height=1100)
fig.show()

In [ ]:
hourly_locations = crash_data.groupby(['Latitude', 'Longitude', 'Hour']).size().reset_index(name='Counts')
hourly_locations = hourly_locations.sort_values(by='Hour')

fig = go.Figure()

for hour in hourly_locations['Hour'].unique():
    filtered_data = hourly_locations[hourly_locations['Hour'] == hour]
    fig.add_trace(go.Densitymapbox(lat=filtered_data['Latitude'],
                                   lon=filtered_data['Longitude'],
                                   z=filtered_data['Counts'],
                                   radius=10,
                                   visible=False, 
                                   name=f'Hour {hour}'))

fig.data[0].visible = True

steps = []
for i, hour in enumerate(hourly_locations['Hour'].unique()):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": f"Crash Density by Hour: {hour}"}],  
    )
    step["args"][0]["visible"][i] = True 
    steps.append(step)

fig.update_layout(updatemenus=[dict(type='buttons', showactive=False,
                                    buttons=[dict(label='Play',
                                                  method='animate',
                                                  args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True)])])])

fig.update_layout(height=800, width=800)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Hour: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    mapbox_style="open-street-map",
    mapbox_center={"lat": crash_data['Latitude'].mean(), "lon": crash_data['Longitude'].mean()},
    mapbox_zoom=10)

fig.show()

In [ ]:
monthly_locations = crash_data.groupby(['Latitude', 'Longitude', 'Month']).size().reset_index(name='Counts')

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='dynamic-crash-map'),
    html.Button('Start/Stop', id='play-button', n_clicks=0),
    dcc.Interval(
            id='interval-component',
            interval=1*1000,  
            n_intervals=0,
            disabled=True  
        )
])

@app.callback(
    Output('interval-component', 'disabled'),
    [Input('play-button', 'n_clicks')],
    [State('interval-component', 'disabled')]
)
def toggle_interval(n_clicks, is_disabled):
    if n_clicks:
        return not is_disabled
    return is_disabled

@app.callback(Output('dynamic-crash-map', 'figure'),
              [Input('interval-component', 'n_intervals')],
              [State('interval-component', 'disabled')])
def update_map(n, is_disabled):
    if is_disabled:
        raise dash.exceptions.PreventUpdate
    
    month = (n % 12) + 1
    
    filtered_data = monthly_locations[monthly_locations['Month'] == month]
    
    fig = go.Figure(go.Densitymapbox(lat=filtered_data['Latitude'], 
                                     lon=filtered_data['Longitude'], 
                                     z=filtered_data['Counts'], 
                                     radius=10,
                                     name=f'Month {month}'))

    
    months_of_year = ['January', 'February', 'March', 'April', 'May', 'June', 
                      'July', 'August', 'September', 'October', 'November', 'December']
    
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_center={"lat": crash_data['Latitude'].mean(), "lon": crash_data['Longitude'].mean()},
        mapbox_zoom=10,
        title=f"Crash Density: {months_of_year[month-1]}",
        width=1000, height=1100
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port = 8051)
